# This notebook is meant to train RNNs with GAN implementation



*********

## Brief architecture and input/output abstract

### Generator :

    The Generator takes a sequence from the database of size sequence_length_in, this sequence goes to a RNN (rnn, lstm or gru), then, the last hidden_state of the rnn goes to a fully connected (linear) layer that generates one chord. This chord is added to the sequence, while the first chord of the sequence is removed. We then repeat this process until we generate a new sequence of length sequence_length_out.
    
### Discriminator :

    The Discriminator takes a sequence of chords ad its input, this sequence goes to a RNN, and ad before, the last layer of the hidden_state of the RNN is used as an input for a linear layer. This linear layer then outputs a scalar. The goal of the discriminator is to output 1 if the input data was from the dataset and 0 if it was generated by the generator. 
    
***

### Note : This is not working yet, somehow the generator do not updates weights while training on the discriminator network


#### Note2 : As it is still in development, some of the parameters bellow are useless and have not been removed

In [2]:
params = {
        'print_every': 5,
        'optimizer': "SGD",
        'lossFunction': "MSE",
        'model_type': "gru",
        'alphabet': 'a0',
        'sequence_lenght_in': 16, 
        'sequence_lenght_out': 16, 
        'using_cuda': True,
        'batch_size': 256,
        'shuffle': True, 
        'num_workers': 6,
        'hidden_size_generator': 512,
        'num_layers_generator': 2,
        'dropout_generator': 0.01,
        'learning_rate_generator': 5e-2, 
        'hidden_size_discriminator':512,
        'num_layers_discriminator':2,
        'dropout_discriminator': 0.01,
        'learning_rate_discriminator': 5e-2, 
        'epochs': 50,
        'use_Paul_distance': True}

paramsGen = {
        'print_every': 5,
        'optimizer': "SGD",
        'lossFunction': "MSE",
        'model_type': "gru",
        'alphabet': 'a0',
        'sequence_lenght_in': 16, 
        'sequence_lenght_out': 16, 
        'using_cuda': True,
        'batch_size': 128,
        'shuffle': True, 
        'num_workers': 6,
        'hidden_size_generator': 512,
        'num_layers_generator': 2,
        'dropout_generator': 0.01,
        'learning_rate_generator': 1e-4, 
        'epochs': 50}

paramsGenOnData = {
        'print_every': 5,
        'plot_every': 5,
        'optimizer': "SGD",
        'lossFunction': "MSE",
        'model_type': "gru",
        'alphabet': 'a0',
        'sequence_lenght': 16, 
        'using_cuda': True,
        'batch_size': 128,
        'shuffle': True, 
        'num_workers': 6,
        'hidden_size': 512,
        'num_layers': 2,
        'dropout': 0.05,
        'learning_rate': 5e-2, 
        'epochs': 50,
        'use_Paul_distance': True}

paramsDisInit = {
        'input_size':25,
        'hidden_size_discriminator':512,
        'num_layers_discriminator':2,
        'dropout_discriminator': 0.01}

paramsDisOnGen = {
        'print_every': 5,
        'optimizer': "SGD",
        'lossFunction': "MSE",
        'model_type': "gru",
        'alphabet': 'a0',
        'sequence_lenght_in': 16, 
        'sequence_lenght_out': 16, 
        'using_cuda': True,
        'batch_size': 128,
        'shuffle': True, 
        'num_workers': 6,
        'hidden_size_discriminator': 512,
        'num_layers_discriminator': 2,
        'dropout_discriminator': 0.01,
        'learning_rate_discriminator': 5e-2, 
        'epochs': 50
}

paramsDisOnData = {
        'print_every': 5,
        'optimizer': "SGD",
        'lossFunction': "MSE",
        'model_type': "gru",
        'alphabet': 'a0',
        'sequence_lenght_in': 16, 
        'sequence_lenght_out': 16, 
        'using_cuda': True,
        'batch_size': 128,
        'shuffle': True, 
        'num_workers': 6,
        'hidden_size_discriminator': 256,
        'num_layers_discriminator': 2,
        'dropout_discriminator': 0.01,
        'learning_rate_discriminator': 5e-2, 
        'epochs': 50
}



from GruGanClass import MYGRU
from RnnGanClass import MYRNN
from LstmGanClass import MYLSTM
from DiscriminatorClass import MYDis
import torch
import os
from utilities import chordUtil
from utilities.chordUtil import *
from utilities import dataImport



dropout_generator = params['dropout_generator']
model_type = params['model_type']
num_layers_generator = params['num_layers_generator']
hidden_size_generator = params['hidden_size_generator']
alphabet = params['alphabet']
sequence_lenght_in = params['sequence_lenght_in']
sequence_lenght_out = params['sequence_lenght_out']
num_layers_discriminator = params['num_layers_discriminator']
hidden_size_discriminator = params['hidden_size_discriminator']

model_string_generator = "models/"+model_type+str(num_layers_generator)+"layers"+str(hidden_size_generator)+"blocks"+alphabet+"alphabet"+str(sequence_lenght_in)+"lenSeq_in"+str(sequence_lenght_out)+"lenSeq_out.pt"
model_string_discriminator = "models/"+model_type+str(num_layers_discriminator)+"layers"+str(hidden_size_discriminator)+"blocks"+alphabet+"alphabet"+str(sequence_lenght_out)+"lenSeq_out.pt"

# Getting alphabet size :
rootname = "inputs/jazz_xlab/"
filenames = os.listdir(rootname)
dictChord, listChord = chordUtil.getDictChord(eval(alphabet))
alphabet_size = len(dictChord)


In [2]:
rootname = "inputs/jazz_xlab/"
filenames = os.listdir(rootname)
dictChord, listChord = chordUtil.getDictChord(eval(alphabet))
print(len(dictChord))
print(dictChord)

25
{'G#:min': 0, 'G:maj': 1, 'E:min': 2, 'A:min': 3, 'A#:maj': 4, 'C:min': 5, 'D:maj': 6, 'D#:maj': 7, 'A:maj': 8, 'F#:maj': 9, 'C:maj': 10, 'C#:min': 11, 'G#:maj': 12, 'C#:maj': 13, 'G:min': 14, 'E:maj': 15, 'F#:min': 16, 'B:maj': 17, 'D:min': 18, 'B:min': 19, 'N': 20, 'D#:min': 21, 'A#:min': 22, 'F:maj': 23, 'F:min': 24}


# Initialising the networks with user's parameters

In [3]:
if model_type == "gru":
    myGenerator = MYGRU(alphabet_size, hidden_size_generator, num_layers_generator, dropout_generator)
elif model_type == "lstm":
    myGenerator = MYLSTM(alphabet_size, hidden_size_generator, num_layers_generator, dropout_generator)
elif model_type == "rnn":
    myGenerator = MYRNN(alphabet_size, hidden_size_generator, num_layers_generator, dropout_generator)
#TODO making the user choose between  gru, lstm and rnn in the parameters
myDis = MYDis(**paramsDisInit)

# Classic training of the generator on Data

### Note : actually this part is not a part of the GAN, but it can be usefull to have a generator somewhat trained in order to train the discriminator correctly

In [6]:
paramsGenOnData['epochs']=10
paramsGenOnData['print_every']=2
myGenerator.trainAndTest(**paramsGenOnData)

Trying using Cuda ...
OK
Start training
Train : 0m 2s (2 20%) loss : 0.4868, accuracy : 12.1336%
Test : 0m 3s (2 20%) loss : 0.1460, accuracy : 14.8269%
Train : 0m 5s (4 40%) loss : 0.4322, accuracy : 14.8593%
Test : 0m 6s (4 40%) loss : 0.1460, accuracy : 16.8590%
Train : 0m 8s (6 60%) loss : 0.4883, accuracy : 21.2192%
Test : 0m 9s (6 60%) loss : 0.1460, accuracy : 19.2173%
Train : 0m 11s (8 80%) loss : 0.5207, accuracy : 20.7503%
Test : 0m 12s (8 80%) loss : 0.1460, accuracy : 19.8695%
Finished Training


# Try training the generator to fool the discriminator

## Problem here !
### Generator does not train :'(

In [7]:
paramsGen['epochs']=5
paramsGen['print_every']=1
paramsGen['learning_rate_generator']=1e-2
paramsGen['dropout_generator']=0.1
paramsGen['disNet']=myDis

myGenerator.trainOnDis(**paramsGen)

Trying using Cuda ...
OK
Start training
genWin : 0m 15s (1 20%) loss : 0.7583, accuracy : 0.0000%
genWin : 0m 22s (2 40%) loss : 0.7581, accuracy : 0.0000%
genWin : 0m 30s (3 60%) loss : 0.7582, accuracy : 0.0000%
genWin : 0m 37s (4 80%) loss : 0.7584, accuracy : 0.0000%
Finished Training


[0.0, 0.0, 0.0, 0.0]

# Training the discriminator to recognize fake Data

In [8]:
paramsDisOnGen['epochs']=5
paramsDisOnGen['print_every']=1
paramsDisOnGen['learning_rate_discriminator']=5e-5
paramsDisOnGen['dropout_discriminator']=0.1
paramsDisOnGen['genNet']=myGenerator

myDis.trainOnGen(**paramsDisOnGen)

Trying using Cuda ...
OK
Start training
disOnGen : 0m 13s (1 20%) loss : 0.0008, accuracy : 100.0000%
disOnGen : 0m 20s (2 40%) loss : 0.0008, accuracy : 100.0000%
disOnGen : 0m 27s (3 60%) loss : 0.0008, accuracy : 100.0000%
disOnGen : 0m 34s (4 80%) loss : 0.0008, accuracy : 100.0000%
Finished Training


[1.0, 1.0, 1.0, 1.0]

# Training the discriminator to recognize real Data

In [9]:
paramsDisOnData['epochs']=5
paramsDisOnData['print_every']=1
paramsDisOnData['learning_rate_discriminator']=2e-5
paramsDisOnData['dropout_discriminator']=0.1

myDis.trainOnData(**paramsDisOnData)

Trying using Cuda ...
OK
Start training
disOnDataWin : 0m 2s (1 20%) loss : 0.7600, accuracy : 0.0000%
disOnDataWin : 0m 3s (2 40%) loss : 0.7550, accuracy : 0.0000%
disOnDataWin : 0m 4s (3 60%) loss : 0.7515, accuracy : 0.0000%
disOnDataWin : 0m 5s (4 80%) loss : 0.7482, accuracy : 0.0000%
Finished Training


[0.0, 0.0, 0.0, 0.0]

# Training the discriminator on Real and Fake Data
## Also using more real data or more fake data dependinig on its weak points

In [10]:
# Train discriminator
disOnGenWin = 1
disOnDataWin = 1
metaEpochs = 5
paramsDisOnGen['epochs']=2
paramsDisOnData['epochs']=2

for i in range(metaEpochs):
    print("\niteration: %d\n" % i)
    if disOnDataWin>0.75 or disOnDataWin>disOnGenWin:
        paramsDisOnGen['epochs']=4
        paramsDisOnData['epochs']=4

    if disOnGenWin>0.75 or disOnGenWin>disOnDataWin:
        paramsDisOnGen['epochs']=2
        paramsDisOnData['epochs']=7
        
    disOnGenWin = max(myDis.trainOnGen(**paramsDisOnGen))
    disOnDataWin = max(myDis.trainOnData(**paramsDisOnData))






iteration: 0

Trying using Cuda ...
OK
Start training
disOnGen : 0m 13s (1 50%) loss : 0.0014, accuracy : 100.0000%
Finished Training
Trying using Cuda ...
OK
Start training
disOnDataWin : 0m 2s (1 14%) loss : 0.7437, accuracy : 0.0000%
disOnDataWin : 0m 3s (2 28%) loss : 0.7391, accuracy : 0.0000%
disOnDataWin : 0m 4s (3 42%) loss : 0.7360, accuracy : 0.0000%
disOnDataWin : 0m 6s (4 57%) loss : 0.7326, accuracy : 0.0000%
disOnDataWin : 0m 7s (5 71%) loss : 0.7292, accuracy : 0.0000%
disOnDataWin : 0m 8s (6 85%) loss : 0.7257, accuracy : 0.0000%
Finished Training

iteration: 1

Trying using Cuda ...
OK
Start training
disOnGen : 0m 13s (1 50%) loss : 0.0025, accuracy : 100.0000%
Finished Training
Trying using Cuda ...
OK
Start training
disOnDataWin : 0m 2s (1 14%) loss : 0.7216, accuracy : 0.0000%
disOnDataWin : 0m 3s (2 28%) loss : 0.7170, accuracy : 0.0000%
disOnDataWin : 0m 4s (3 42%) loss : 0.7137, accuracy : 0.0000%
disOnDataWin : 0m 5s (4 57%) loss : 0.7100, accuracy : 0.0000%
di

# Train both generator and discriminator
## note : this algorithm is certainly really bad, as the generator does not train well for now, I cannot improve it efficiently

In [11]:
#Train Global
paramsGen['epochs']=2
paramsDisOnGen['epochs']=2
paramsDisOnData['epochs']=2

metaEpochs = 5

genWin = 0
disOnGenWin = 0
disOnDataWin = 0

for i in range(metaEpochs):
    if disOnGenWin < 0.75:
        disOnGenWin = max(myDis.trainOnGen(**paramsDisOnGen))
    if disOnDataWin < 0.75:
        disOnDataWin = max(myDis.trainOnData(**paramsDisOnData))
    else:
        genWin = max(myGenerator.trainOnDis(**paramsGen))

print("finished")



Trying using Cuda ...
OK
Start training
disOnGen : 0m 13s (1 50%) loss : 0.0099, accuracy : 100.0000%
Finished Training
Trying using Cuda ...
OK
Start training
disOnDataWin : 0m 2s (1 50%) loss : 0.6410, accuracy : 0.0000%
Finished Training
Trying using Cuda ...
OK
Start training
disOnDataWin : 0m 2s (1 50%) loss : 0.6353, accuracy : 0.0000%
Finished Training
Trying using Cuda ...
OK
Start training
disOnDataWin : 0m 2s (1 50%) loss : 0.6292, accuracy : 0.0000%
Finished Training
Trying using Cuda ...
OK
Start training
disOnDataWin : 0m 2s (1 50%) loss : 0.6234, accuracy : 0.0000%
Finished Training
Trying using Cuda ...
OK
Start training
disOnDataWin : 0m 2s (1 50%) loss : 0.6177, accuracy : 0.0000%
Finished Training
finished


# Testing the generator on some simple chord sequence

In [12]:
test_sequence = ["C:maj","F:maj","C:maj","C:maj",
                 "F:maj","F:maj","C:maj","C:maj",
                 "G:maj","F:maj","C:maj","G:maj",
                 "C:maj","C:maj","F:maj","G:maj"]
myGenerator.generateFromSequence(test_sequence, generation_lenght=64,
                               alphabet = 'a0',sampling=True, 
                               using_cuda=True, silent=True)

Trying using Cuda ...
OK
['C:maj', 'F:maj', 'C:maj', 'C:maj']
['F:maj', 'F:maj', 'C:maj', 'C:maj']
['G:maj', 'F:maj', 'C:maj', 'G:maj']
['C:maj', 'C:maj', 'F:maj', 'G:maj']
generated :
['E:maj', 'C#:maj', 'A:maj', 'N']
['F:min', 'F:maj', 'C#:maj', 'D#:maj']
['F:min', 'A#:maj', 'C:maj', 'G#:maj']
['A#:min', 'B:maj', 'E:min', 'G#:min']
['D:maj', 'A#:min', 'F:maj', 'B:min']
['E:min', 'A:maj', 'D:maj', 'C#:min']
['F:min', 'F#:min', 'A:min', 'F#:maj']
['G#:min', 'C#:maj', 'C:maj', 'F#:min']
['D:min', 'E:min', 'E:min', 'C:min']
['D#:min', 'A#:maj', 'D:min', 'G:min']
['E:maj', 'F#:min', 'A#:min', 'G:maj']
['F#:maj', 'G#:maj', 'G:min', 'E:min']


# Testing if the discriminator tells that the test sequence is real or false (1 is true and 0 is false)

In [13]:
using_cuda = True
# Cuda blabla
if using_cuda:
    print("Trying using Cuda ...")
    use_cuda = torch.cuda.is_available()
    if use_cuda:
        print("OK")
    else:
        print("Woops, Cuda cannot be found :'( ")
    device = torch.device("cuda:0" if use_cuda else "cpu")
else:
    device = torch.device("cpu")
    print("Using Cpu")

lenSeq = len(test_sequence)


# Getting chords dictionary
rootname = "inputs/jazz_xlab/"
filenames = os.listdir(rootname)
dictChord, listChord = chordUtil.getDictChord(eval(alphabet))


# Initialising objects
test_sequence_tensor = torch.zeros(1, len(test_sequence), len(dictChord)).to(device)
last_chords_output = torch.zeros(1, lenSeq, len(dictChord)).to(device)
test_sequence_tensor.requires_grad = False
last_chords_output.requires_grad = False
for t in range(len(test_sequence)):
    test_sequence_tensor[0, t, dictChord[test_sequence[t]]] = 1
    

myDis(test_sequence_tensor)

Trying using Cuda ...
OK


tensor([[0.1244]], device='cuda:0', grad_fn=<ThAddmmBackward>)